# Basic settings

In [1]:
(ext:chdir "~/Development/clasp/extensions/cando/src/tests/boron/" t)

0

#### Create all paths.

In [2]:
(leap.core::clear-path)
(add-path "~/Development/clasp/extensions/cando/src/tests/boron/")
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/"
 #P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/boron/")

In [3]:
(start-swank)

swank-loader -> /Users/tuj24515/Development/slime/swank-loader.lisp

"Read: (DEFPACKAGE :SWANK-LOADER (:USE :CL) (:EXPORT :INIT :DUMP-IMAGE :LIST-FASLS :*SOURCE-DIRECTORY* :*FASL-DIRECTORY*))
" 
"Read: (IN-PACKAGE :SWANK-LOADER)
" 
"Read: (DEFVAR *SOURCE-DIRECTORY* (MAKE-PATHNAME :NAME NIL :TYPE NIL :DEFAULTS (OR *LOAD-PATHNAME* *DEFAULT-PATHNAME-DEFAULTS*)) The directory where to look for the source.)
" 
"Read: (DEFPARAMETER *SYSDEP-FILES* (QUOTE ((SWANK CLASP) (SWANK GRAY))))
" 
"Read: (DEFPARAMETER *IMPLEMENTATION-FEATURES* (QUOTE (:ALLEGRO :LISPWORKS :SBCL :CLOZURE :CMU :CLISP :CCL :CORMAN :CORMANLISP :ARMEDBEAR :GCL :ECL :SCL :MKCL :CLASP)))
" 
"Read: (DEFPARAMETER *OS-FEATURES* (QUOTE (:MACOSX :LINUX :WINDOWS :MSWINDOWS :WIN32 :SOLARIS :DARWIN :SUNOS :HPUX :UNIX)))
" 
"Read: (DEFPARAMETER *ARCHITECTURE-FEATURES* (QUOTE (:POWERPC :PPC :X86 :X86-64 :X86_64 :AMD64 :I686 :I586 :I486 :PC386 :IAPX386 :SPARC64 :SPARC :HPPA64 :HPPA :ARM :ARMV5L :ARMV6L :ARMV7L :ARM64 :PENTIUM3 :PENTIUM4 

T

#### Load the GAFF force field and boronic ester frcmod file.

In [4]:
(clear-force-field)
(load-atom-type-rules "ATOMTYPE_GFF.DEF")

(#<FORCE-FIELD ATOMTYPE_GFF.DEF>)

In [5]:
(load-amber-params "gaff.dat")

SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


#<FORCE-FIELD gaff.dat>

In [6]:
(load-atom-type-rules "boron.def")
(load-atom-type-rules "water_gaff.def")

(#<FORCE-FIELD water_gaff.def> #<FORCE-FIELD boron.def> #<FORCE-FIELD gaff.dat>
 #<FORCE-FIELD ATOMTYPE_GFF.DEF>)

In [7]:
(load-amber-params "frcmod.boronic-ester")
(gethash :default leap.core::*force-fields*)

(#<FORCE-FIELD frcmod.boronic-ester> #<FORCE-FIELD water_gaff.def>
 #<FORCE-FIELD boron.def> #<FORCE-FIELD gaff.dat>
 #<FORCE-FIELD ATOMTYPE_GFF.DEF>)

#### Merge force field and define parameter ff.

In [8]:
(:= *ff* (leap.core::merged-force-field))

SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb


*FF*

## Boronic ester 01

#### Load the arylboronic ester from a Chemdraw file.

In [9]:
(defparameter *b01agg* (cando:load-chem-draw-aggregate "boronic-ester01.cdxml"))

*B01AGG*

In [10]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *b01agg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *b01agg*)))

C_108  S
C_110  S
C_133  S
C_135  S
C_1968 S
C_1969 S
C_1970 S
C_1981 S
C_1982 S
C_2003 S
C_2004 S
C_2005 S
C_2016 S
C_2017 S
C_208  S
C_210  S
C_233  S
C_235  S
C_722  S
C_723  S
C_724  S
C_725  S
C_726  S
Set atom C_110 to :S
Set atom C_135 to :S
Set atom C_210 to :S
Set atom C_235 to :S


NIL

#### Minimize the energy of the molecule.

In [11]:
(chem:assign-types *ff* *b01agg*)

NIL

In [12]:
(jostle *b01agg*)

NIL

In [13]:
(defparameter *b01e* (energy:minimize *b01agg* :force-field *ff* :restraints-on t))

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000         177696.538
 minSDnP       0     2  0.000447   0.0     1583956364.989          91177.166
 minSDnP       0     3  0.000351   0.0      649480552.895          48030.780
 minSDnP       0     4  0.000401   0.0      351500478.698          34648.313
 minSDnP       0     5  0.000335   0.0      224827489.834          23406.042
 minSDnP       0     6  0.000387   0.0      155164728.057          19840.132
 minSDnP       0     7  0.000332   0.0      115387582.986          14980.233
 minSDnP       0     8  0.000401   0.0       85386199.307          13525.178
 minSDnP       0     9  0.000343   0.0       66487927.270          10595.476
 minSDnP       0    10  0.000398   0.0       50861501.664           9960.129
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0    11  0.000331

*B01E*

In [14]:
(nglv::show-aggregate *b01agg*)

A Jupyter Widget

#### Save the topology file and coordinate file.

In [15]:
(chem:set-property *b01agg* :bounding-box '(30.0 30.0 30.0))

NIL

In [16]:
(chem:assign-types *ff* *b01agg*)

NIL

In [17]:
(leap.topology:save-amber-parm-format *b01agg* "b01.top" "b01.crd" *ff* :assign-types nil)

atom1: #<ATOM :C_724/:C> atom2: #<ATOM :H/:H> i1: 3 i2: 0
 atom1: #<ATOM :C_724/:C> atom2: #<ATOM :C_723/:C> i1: 3 i2: 12
 atom1: #<ATOM :C_725/:C> atom2: #<ATOM :C_724/:C> i1: 6 i2: 3
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C_724/:C> i1: 9 i2: 3
 atom1: #<ATOM :C_723/:C> atom2: #<ATOM :C_722/:C> i1: 12 i2: 27
 atom1: #<ATOM :C_723/:C> atom2: #<ATOM :OH/:O> i1: 12 i2: 30
 atom1: #<ATOM :C_726/:C> atom2: #<ATOM :C_725/:C> i1: 15 i2: 6
 atom1: #<ATOM :C_726/:C> atom2: #<ATOM :O/:O> i1: 15 i2: 36
 atom1: #<ATOM :C_726/:C> atom2: #<ATOM :H/:H> i1: 15 i2: 39
 atom1: #<ATOM :C_726/:C> atom2: #<ATOM :C_735/:C> i1: 15 i2: 42
 atom1: #<ATOM :H/:H> atom2: #<ATOM :C_725/:C> i1: 18 i2: 6
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :C_725/:C> i1: 21 i2: 6
 atom1: #<ATOM :B/:B> atom2: #<ATOM :C_310/:C> i1: 24 i2: 45
 atom1: #<ATOM :B/:B> atom2: #<ATOM :OH/:O> i1: 24 i2: 48
 atom1: #<ATOM :B/:B> atom2: #<ATOM :OH/:O> i1: 24 i2: 51
 atom1: #<ATOM :B/:B> atom2: #<ATOM :O/:O> i1: 24 i2: 9
 atom1: #<ATOM :C_722/:C

0

#### Calculate each energy parameters.

In [18]:
(chem:get-energy (chem:get-stretch-component *b01e*))

84.61729451288969d0

In [19]:
(chem:get-energy (chem:get-angle-component *b01e*))

103.70178925190395d0

In [20]:
(chem:get-energy (chem:get-dihedral-component *b01e*))

188.72176708710498d0

In [21]:
(chem:get-energy (chem:get-nonbond-component *b01e*))

247.71592674541043d0

#### Create Tip3 water molecules box.

In [22]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

In [23]:
(nglv::show-aggregate ~tip3pfbox)

A Jupyter Widget

In [24]:
(print ~tip3pfbox)


AGGREGATE 

AGGREGATE

In [25]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [26]:
(chem:assign-types *ff* *solvent*)

NIL

#### Add Tip3 water molecules to the boronic ester.

In [27]:
(:= *b01wagg* (leap.solvate::tool-solvate-and-shell *b01agg* *solvent* '(30.0 30.0 30.0) :shell t :farness 7.0))

Total bounding box for atom centers:  82.80921107132514d0 85.22909737539987d0 67.1958663054946d0


*B01WAGG*

In [28]:
(chem:map-atoms 'list (lambda (a) (chem:get-type a)) *b01wagg*)

(:|h1| :|c3| :|c3| :|os| :|c3| :|c3| :|h1| :|oh| :|b| :|c3| :|oh| :|h1| :|os|
 :|h1| :|c3| :|c2| :|oh| :|oh| :|h2| :|os| :|oh| :|c2| :|c2| :|c3| :|c2| :|c2|
 :|c3| :|c3| :|h1| :|c2| :|os| :|h1| :|c3| :|os| :|h1| :|c3| :|c3| :|b| :|c3|
 :|h1| :|oh| :|oh| :|n| :|c2| :|oh| :|oh| :|h2| :|os| :|c3| :|c| :|c2| :|c2|
 :|c3| :|c3| :|c3| :|c| :|c3| :|o| :|c2| :|c2| :|c3| :|c3| :|h1| :|c3| :|n3|
 :|o| :|n| :|c3| :|c2| :|os| :|h1| :|c3| :|os| :|h1| :|c3| :|c| :|c3| :|c3|
 :|c3| :|b| :|c3| :|h1| :|oh| :|oh| :|o| :|oh| :|n| :|c| :|c2| :|oh| :|oh|
 :|h2| :|os| :|c| :|o| :|n| :|c2| :|c2| :|b| :|c3| :|o| :|c3| :|c2| :|c2| :|c2|
 :|oh| :|oh| :|c3| :|c3| :|c2| :|c2| :|c2| :|n| :|c| :|c3| :|c2| :|c2| :|c|
 :|o| :|n| :|c2| :|c3| :|o| :|c3| :|c3| :|c3| :|c3| :|n| :|c| :|c| :|n| :|o|
 :|o| :|c3| :|c2| :|c2| :|c2| :|c2| :|c2| :|c2| :|ha| :|ha| :|ha| :|ha| :|ha|
 :|h1| :|h1| :|hc| :|hc| :|h1| :|h1| :|h1| :|h1| :|h1| :|ha| :|ha| :|h1| :|h1|
 :|ha| :|ha| :|hc| :|hc| :|ha| :|ha| :|h1| :|h1| :|h1| :|ha| :|ha| :|h

#### Minimize the energy of the water and boronic ester.

In [29]:
(defparameter *b01we* (energy:minimize *b01wagg* :force-field *ff* :restraints-on t))

======= Starting Steepest Descent Minimizer
DONE absolute force test:
forceRmsMag(5.320690).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minCGnP       0     1  0.000000   0.0              0.000              5.321
 minCGnP       0     2  0.000674  31.0            341.424              3.202
 minCGnP       0     3  0.000918  46.6            323.894              2.893
 minCGnP       0     4  0.000764  40.1            311.981              1.648
 minCGnP       0     5  0.000852  45.6            307.668              1.286
 minCGnP       0     6  0.001083  56.4            304.330              1.350
 minCGnP       0     7  0.001010  62.5            300.895              1.436
 minCGnP       0     8  0.001168  64.8            296.404              1.417
 minCGnP       0     9  0.000894  62.0            293.060              1.134
 minCGnP       0    10  0.000986  65.7            290.696    

*B01WE*

In [30]:
(nglv::show-aggregate *b01wagg*)

A Jupyter Widget

#### Save the topology file and coordinate file.

In [31]:
(chem:assign-types *ff* *b01wagg*)

NIL

In [32]:
(leap.topology:save-amber-parm-format *b01wagg* "b01_water.top" "b01_water.crd" *ff* :assign-types nil)

atom1: #<ATOM :C_724/:C> atom2: #<ATOM :H/:H> i1: 3 i2: 0
 atom1: #<ATOM :C_725/:C> atom2: #<ATOM :C_724/:C> i1: 6 i2: 3
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C_724/:C> i1: 9 i2: 3
 atom1: #<ATOM :C_723/:C> atom2: #<ATOM :C_724/:C> i1: 12 i2: 3
 atom1: #<ATOM :C_726/:C> atom2: #<ATOM :C_725/:C> i1: 15 i2: 6
 atom1: #<ATOM :H/:H> atom2: #<ATOM :C_725/:C> i1: 18 i2: 6
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :C_725/:C> i1: 21 i2: 6
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :HH/:H> i1: 21 i2: 639
 atom1: #<ATOM :B/:B> atom2: #<ATOM :OH/:O> i1: 24 i2: 48
 atom1: #<ATOM :B/:B> atom2: #<ATOM :OH/:O> i1: 24 i2: 51
 atom1: #<ATOM :B/:B> atom2: #<ATOM :O/:O> i1: 24 i2: 9
 atom1: #<ATOM :C_722/:C> atom2: #<ATOM :C_723/:C> i1: 27 i2: 12
 atom1: #<ATOM :C_722/:C> atom2: #<ATOM :H/:H> i1: 27 i2: 54
 atom1: #<ATOM :C_722/:C> atom2: #<ATOM :O/:O> i1: 27 i2: 57
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :C_723/:C> i1: 30 i2: 12
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :HH/:H> i1: 30 i2: 636
 atom1: #<ATOM :H/:H> atom2: #

0

In [33]:
(chem:get-energy (chem:get-stretch-component *b01we*))

85.20410241550098d0

In [34]:
(chem:get-energy (chem:get-angle-component *b01we*))

96.07150351512205d0

In [35]:
(chem:get-energy (chem:get-dihedral-component *b01we*))

186.88429489329016d0

In [36]:
(chem:get-energy (chem:get-nonbond-component *b01we*))

-214.11550168959332d0